# Testing gwit

This notebook is aimed at implementing a gwit client by following the specification defined here:
https://git.sr.ht/~ivilata/gwit-spec. The gwit sites used in this code are "The Oldest gwit Site" (0x408198c2c363076c6b1eabe797ea3168a78cd65a), available at https://git.sr.ht/~ivilata/oldest-gwit-site, and Matograine's site (0x16c8a566bb88303c2513cf6328996d46e0440e85), available at https://framagit.org/matograine/gwitsite


In [1]:
!pip install gitpython pgpy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pgpy
from git import Repo

In [7]:
site_id = "408198c2c363076c6b1eabe797ea3168a78cd65a"
repo_url = "https://git.sr.ht/~ivilata/oldest-gwit-site"
output_path = f"/tmp/{site_id}"
branch = f"gwit-0x{site_id[-8:]}"

print(f"We will be cloning branch `{branch}` from {repo_url}")

We will be cloning branch `gwit-0xa78cd65a` from https://git.sr.ht/~ivilata/oldest-gwit-site


### 1. Clone the Git repository from the given location into temporary storage

e.g. `git clone --bare --branch <SITE-BRANCH> <SITE-LOCATION> <TEMP-REPO> && cd <TEMP-REPO>`


In [8]:
if not os.path.exists(output_path):
    try:
        repo = Repo.clone_from(repo_url, output_path, branch=branch)
    except Exception as e:
        print(e)
else:
    print(f"[x] Output path {output_path} already exists")
    repo = Repo(output_path)

[x] Output path /tmp/408198c2c363076c6b1eabe797ea3168a78cd65a already exists


In [9]:
!ls -l {output_path}

total 16
drwxr-xr-x  3 mala  wheel    96 Feb 25 00:40 LICENSES
drwxr-xr-x  6 mala  wheel   192 Feb 25 03:13 _gwit
-rw-r--r--  1 mala  wheel   231 Feb 25 00:40 index.gmi
-rw-r--r--  1 mala  wheel  1072 Feb 25 00:40 index__en.gmi
drwxr-xr-x  3 mala  wheel    96 Feb 25 00:40 log


### 2. Get the commit at the head of the site branch as `<HEAD-COMMIT>`

e.g. `git show-ref --verify --hash refs/heads/<SITE-BRANCH>`

In [10]:
# ! cd {output_path} && git show-ref --verify --hash refs/heads/{branch}
head_commit_hash = repo.head.commit.hexsha
head_commit_hash

'a805b3caf323e6e9aad6242c15b2316d73b94b61'

### 3. Check that `self.key` exists as a file (blob) in the `_gwit` directory of `<HEAD-COMMIT>`

e.g. `git ls-tree --format='%(objecttype) %(objectname)' <HEAD-COMMIT> _gwit/self.key reports blob <KEY-FILE-HASH>)`


In [11]:
# ! cat /tmp/408198c2c363076c6b1eabe797ea3168a78cd65a/_gwit/self.key
# ! cd {output_path} && git ls-tree --format='%(objecttype) %(objectname)' a805b3caf323e6e9aad6242c15b2316d73b94b61 _gwit/self.key

# get hash of the head commit
commit = repo.commit(head_commit_hash)

# Get the blob hash for the gwit site pub key
try:
    blob = commit.tree / '_gwit/self.key'
    # Get the hash of the blob
    blob_hash = blob.hexsha
    print(f"[i] {blob_hash}")
except KeyError as e:
    print("[x] File is not there")

[i] ee3c8da562f4fc18390d061619403c5c2a87dde0


### 4.. Check that the fingerprint of the primary PGP key in `_gwit/self.key` is equal to `<SITE-ID>` (case-insensitively)
e.g. `git cat-file blob <KEY-FILE-HASH> | gpg --show-keys --with-fingerprint --with-colons | grep -A1 '^pub:' | grep -qiE '^fpr:+<SITE-ID>:$')`

In [12]:
# ! cd {output_path} && git cat-file blob ee3c8da562f4fc18390d061619403c5c2a87dde0 | gpg --show-keys --with-fingerprint --with-colons
import pgpy 

# Get the blob from its hash
blob = repo.git.get_object_data(blob_hash)
public_key_pem = blob[3]

# Parse the armored key
public_key, _ = pgpy.PGPKey.from_blob(public_key_pem)

# Get the fingerprint
fingerprint = public_key.fingerprint
print(fingerprint)
print(site_id)
assert(fingerprint.lower()==site_id.lower())

408198C2C363076C6B1EABE797EA3168A78CD65A
408198c2c363076c6b1eabe797ea3168a78cd65a


In [13]:
# # NOTE: this might be easier to implement in micropython 
# # if we did not use gnupg. Gnupg has armored PEM we cannot
# # directly read with the following code (that works with 
# # my RSA public key instead generated with
# # openssl rsa -in ~/.ssh/id_rsa -pubout -out id_rsa.pub.pem)
# # Leaving this code as possible example (we might want to
# # implement different signing / fingerprinting algos for
# # micropython using ucrypto, see e.g.:
# # https://github.com/dmazzella/ucrypto
# # https://stackoverflow.com/questions/64733471/how-to-calculate-a-fingerprint-from-an-rsa-public-key

# from cryptography.hazmat.primitives import hashes
# from cryptography.hazmat.primitives.asymmetric import rsa
# from cryptography.hazmat.primitives import serialization

# # fname = '/tmp/408198c2c363076c6b1eabe797ea3168a78cd65a/_gwit/self.key'
# fname = "/tmp/408198c2c363076c6b1eabe797ea3168a78cd65a/id_rsa.pub.pem"

# def get_fingerprint(public_key_pem):
#     public_key = serialization.load_pem_public_key(public_key_pem)
#     public_bytes = public_key.public_bytes(
#         encoding=serialization.Encoding.DER,
#         format=serialization.PublicFormat.SubjectPublicKeyInfo
#     )
#     digest = hashes.Hash(hashes.SHA256())
#     digest.update(public_bytes)
#     return digest.finalize()

# with open(fname, 'rb') as key_file:
#     public_key_pem = key_file.read()

# fingerprint = get_fingerprint(public_key_pem)
# print(fingerprint.hex())


### 5. Import `_gwit/self.key` into the client's keyring

e.g. `git cat-file blob <KEY-FILE-HASH> | gpg --homedir <CLIENT-GPG-DIR> --import`

... this can be done manually but is not required if we dont' use gnupg (see the two solutions below)

### 6. Check that `<HEAD-COMMIT>` has a valid signature by the key that matches `<SITE-ID>` (case-insensitively), or by a subkey of it 

e.g. `git verify-commit --raw <HEAD-COMMIT> 2>&1 | sed -nE 's/^\[GNUPG:\] VALIDSIG .*\b(\S+)$/\1/p' reports <SITE-ID>)`

#### 6.1 Solution 1: call `git verify-commit`, which then calls `gpg`

- pros: works out of the box, git knows how to split payload and public key for verification
- cons: depends on git + gnupg, relies on some string matching

In [14]:
import subprocess

result = subprocess.run(['git', '-C', repo.working_dir, 'show', '--pretty=raw', '--show-signature', head_commit_hash],
                        capture_output=True, text=True)

print(result.stdout[:1000], "...")
assert("gpg: Good signature" in result.stdout)

commit a805b3caf323e6e9aad6242c15b2316d73b94b61
gpg: Signature made Gio 22 Feb 17:31:07 2024 GMT
gpg:                using RSA key 408198C2C363076C6B1EABE797EA3168A78CD65A
gpg: Good signature from "Degauss (The Oldest gwit Site) <degauss+togs@selidor.net>" [unknown]
gpg: WARNING: This key is not certified with a trusted signature!
gpg:          There is no indication that the signature belongs to the owner.
Primary key fingerprint: 4081 98C2 C363 076C 6B1E  ABE7 97EA 3168 A78C D65A
tree 5abe7e37a503a299d741968c19f71898d327b8ab
parent 33f9e1c225f154231c0d7a50e95e5bdb4ccd899e
author Degauss (The Oldest gwit Site) <degauss+togs@selidor.net> 1708623060 +0100
committer Degauss (The Oldest gwit Site) <degauss+togs@selidor.net> 1708623060 +0100
gpgsig -----BEGIN PGP SIGNATURE-----
 
 iQGzBAABCAAdFiEEQIGYwsNjB2xrHqvnl+oxaKeM1loFAmXXhNsACgkQl+oxaKeM
 1lpedwv/RCFPl14yOjKVOwKXWvus2uiWqUDQOWCKaoqavI52i5DtXpYuk3hA01AM
 +jjNLFVmfeJt/gpYaNLeIx5KD79jj5d3A8ElkA5iU6FvQT7y6r2wcq6P0hz4nyqN
 adMRhnEgPDVnrV

In [15]:
# # with open("/tmp/408198c2c363076c6b1eabe797ea3168a78cd65a/_gwit/fuffa", 'rb') as msg_file:
# with open("/tmp/fineeeh", 'rb') as msg_file:
# # with open("/tmp/test_gpg_signed_repo/fuffa", 'rb') as msg_file:
#     message = msg_file.read()

# sig = """-----BEGIN PGP SIGNATURE-----

# iQGzBAABCAAdFiEEQIGYwsNjB2xrHqvnl+oxaKeM1loFAmXXhNsACgkQl+oxaKeM
# 1lpedwv/RCFPl14yOjKVOwKXWvus2uiWqUDQOWCKaoqavI52i5DtXpYuk3hA01AM
# +jjNLFVmfeJt/gpYaNLeIx5KD79jj5d3A8ElkA5iU6FvQT7y6r2wcq6P0hz4nyqN
# adMRhnEgPDVnrV0nT/T+kq4+JBMIxx1ogXOpmLMFIkb2LaK3ra8gYoDfBx0R4IWb
# quGdtrtJoEDYyIG6xXtZ/i580lREO2L2xcfvMMY7ZuZdxCFDF2FvlsjZZwDFQBtg
# 3NuiL2kcjyP1KYL4H09Ea99gpuWbDIo/7VfMJs8XqNneAMNkkG2sirFD1Y1UsBLU
# 2Htaw9cskD2HqmLqh8sPKZrZee3Hndppwu8qri1UL+krR+P/uuYgbCapxftfhSdP
# dEPtwQj3LGKwVNSu699tEated0+RnpK8AgbNviGWUYNFefflfYDyEbQcT68aGjl9
# 7DrbbTMTzxSD+dD5zt1+dhFcnN7XqCntiZOmNbeWqyzTZit0/O8v5lwjzjEi0ldh
# 43cz7496
# =IjTS
# -----END PGP SIGNATURE-----
# """

# sig = pgpy.PGPSignature.from_blob(sig)

# # key = pgpy.PGPKey.from_file("/tmp/test_gpg_signed_repo/gwit.asc")[0]
# key = pgpy.PGPKey.from_file("/tmp/408198c2c363076c6b1eabe797ea3168a78cd65a/_gwit/self.key")[0]

# print(message)

# print(key.verify(message, sig))
# print(key.verify(message.strip(), sig))

#### 6.2 Solution 2: implement everything in gitpython + pgpy

- pros: fewer dependencies, more control
- cons: we need to rebuild the payload from scratch

To test whether the payload makes sense, we'll verify its signature on all the commits in the repo. Let us start with the first one...

In [18]:
def get_commit_payload(commit):
    headers  = f"tree {commit.tree.hexsha}\n"
    try:
        headers += f"parent {commit.parents[0].hexsha}\n"
    except Exception as e:
        print(e)
    headers += f"author {commit.author.name} <{commit.author.email}> {commit.authored_date} {commit.authored_datetime.strftime('%z')}\n"
    try:
        headers += f"committer {commit.committer.name} <{commit.committer.email}> {commit.committed_date} {commit.committed_datetime.strftime('%z')}\n\n"
    except Exception as e:
        print(e)
    headers += commit.message
    headers.encode()
    return headers

# get the payload
payload = get_commit_payload(commit)
# get the signature
sig = pgpy.PGPSignature.from_blob(commit.gpgsig)
# verify 
print(public_key.verify(payload, sig))

<SignatureVerification(True)>


/Users/mala/envs/jup/lib/python3.9/site-packages/pgpy/pgp.py:2389: UserWarning: TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!
  warnings.warn("TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!")
/Users/mala/envs/jup/lib/python3.9/site-packages/pgpy/pgp.py:2406: UserWarning: TODO: Revocation checks are not yet implemented!!!
  warnings.warn("TODO: Revocation checks are not yet implemented!!!")
/Users/mala/envs/jup/lib/python3.9/site-packages/pgpy/pgp.py:2407: UserWarning: TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!
  warnings.warn("TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!")


In [19]:
for c in repo.iter_commits():
    print(f"Commit {c.hexsha}")
    payload = get_commit_payload(commit)
    sig = pgpy.PGPSignature.from_blob(commit.gpgsig)
    print(public_key.verify(payload, sig))

Commit a805b3caf323e6e9aad6242c15b2316d73b94b61
<SignatureVerification(True)>
Commit 33f9e1c225f154231c0d7a50e95e5bdb4ccd899e
<SignatureVerification(True)>
Commit 29b2c1071b8da8815eb8ae32140b635069e831cf
<SignatureVerification(True)>
Commit 080d577d21cd6604ec11bbb12e08c4a29ba2e08e
<SignatureVerification(True)>
Commit ddc36fcdd84dca79a45e180e15fa784d86cc5d86
<SignatureVerification(True)>
Commit 746d02633c21437096ba235e3ab0fb1fdf866c1d
<SignatureVerification(True)>
Commit 49eb4b0a5e3e60ee6e5d4b1c81ade9b9f2ff8c1c
<SignatureVerification(True)>
Commit 9259b51eac0fd029158cd39a73f5901a9befc3a7
<SignatureVerification(True)>
Commit 9d797f4480adce12d53795d9aa0586386db18bff
<SignatureVerification(True)>
Commit abb299bb34a3cc0e317faedda3f91929c3401c08
<SignatureVerification(True)>
Commit ae01109da5c0cdf530b0094c0a010945e2931781
<SignatureVerification(True)>
Commit c7959c006b378dafb4766f66a56411a3ee96431e
<SignatureVerification(True)>
Commit 10fa46abcfc70cb4bb50a8f2565cd86ef176bd7a
<SignatureVerifi

/Users/mala/envs/jup/lib/python3.9/site-packages/pgpy/pgp.py:2389: UserWarning: TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!
  warnings.warn("TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!")
/Users/mala/envs/jup/lib/python3.9/site-packages/pgpy/pgp.py:2406: UserWarning: TODO: Revocation checks are not yet implemented!!!
  warnings.warn("TODO: Revocation checks are not yet implemented!!!")
/Users/mala/envs/jup/lib/python3.9/site-packages/pgpy/pgp.py:2407: UserWarning: TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!
  warnings.warn("TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!")
